In [ ]:
from discretize import TensorMesh
from discretize.utils import mkvc, active_from_xyz

from SimPEG.electromagnetics.static import resistivity as dc, utils as DCutils
from SimPEG.electromagnetics.static.utils.static_utils import (
    generate_dcip_sources_line,
    apparent_resistivity_from_voltage,
    plot_pseudosection
    
)
from SimPEG import (
    maps,
    data_misfit,
    regularization,
    optimization,
    inverse_problem,
    inversion,
    directives,
    utils,
    data
)

import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import warnings 
import matplotlib.gridspec as gridspec
import ipywidgets

try:
    from pymatsolver import Pardiso as Solver
except ImportError:
    from SimPEG import SolverLU as Solver


write_output = False

# mpl.rcParams.update({"font.size": 16})
plt.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True)

In [ ]:
class SaveInversionProgress(directives.InversionDirective):
    """
    A custom directive to save items of interest during the course of an inversion
    """
    
    def initialize(self):
        """
        This is called when we first start running an inversion
        """
        # initialize an empty dictionary for storing results 
        self.inversion_results = {
            "iteration":[],
            "beta":[],
            "phi_d":[],
            "phi_m":[],
            "phi_m_small":[],
            "phi_m_smooth_x":[],
            "phi_m_smooth_z":[],
            "dpred":[],
            "model":[]
        }

    def endIter(self):
        """
        This is run at the end of every iteration. So here, we just append 
        the new values to our dictionary
        """
        
        # Save the data
        self.inversion_results["iteration"].append(self.opt.iter)
        self.inversion_results["beta"].append(self.invProb.beta)
        self.inversion_results["phi_d"].append(self.invProb.phi_d)
        self.inversion_results["phi_m"].append(self.invProb.phi_m)
        self.inversion_results["dpred"].append(self.invProb.dpred)
        self.inversion_results["model"].append(self.invProb.model)
        
        # grab the components of the regularization and evaluate them here
        # the regularization has a list of objective functions  
        # objfcts = [smallness, smoothness_x, smoothness_z]
        # and the multipliers contain the alpha values
        # multipliers = [alpha_s, alpha_x, alpha_z]
        reg = self.reg.objfcts[0] 
        phi_s = reg.objfcts[0](self.invProb.model) * reg.multipliers[0]
        phi_x = reg.objfcts[1](self.invProb.model) * reg.multipliers[1]
        phi_z = reg.objfcts[2](self.invProb.model) * reg.multipliers[2]
        
        self.inversion_results["phi_m_small"].append(phi_s)
        self.inversion_results["phi_m_smooth_x"].append(phi_x)
        self.inversion_results["phi_m_smooth_z"].append(phi_z)


In [ ]:
def Forward(model=None, surveyinfo = None):

    if model is None:
        warnings.warn('Warning Message: Ingrese el modelo, mi perro')

    if surveyinfo is None:
        warnings.warn('Warning Message: Ingrese el la información del survey, mi perro. ¿Qué pasa, mi rey?')

    model = model[::-1]
    nrow, ncol = model.shape
    nElec = surveyinfo["nElec"]
    sep = surveyinfo["sep"]

    pi = surveyinfo["pi"] # punto inicial 
    pf = sep*nElec # punto final

    # Define survey line parameters
    survey_type = surveyinfo["typ_survey"] # Tipo arreglo
    dimension_type = surveyinfo["dim"]
    data_type = surveyinfo["data_type"] # Tipo de datos: voltajes, resistividades, conductividades

    end_locations = np.r_[pi, pf] # Posiciones de los electrodos (xi, xf)
    station_separation = sep # Separación 
    num_rx_per_src = surveyinfo["nlines"] # Número máximo de líneas (n)

    # Definición de topografía en función del arreglo
    x = np.linspace(pi, pf, int(pf+1)) # el numero de elementos debe ser > a los elementos en el eje X de la malla
    z = np.linspace(-pf/3, 0, int(pf/3)+1)
    x_topo, z_topo = np.meshgrid(x, z)
    topo_xz = np.c_[mkvc(x_topo), mkvc(z_topo)]

    topo_2d = np.unique(topo_xz[:, [0, 1]], axis=0)

    # Generate source list for DC survey line
    source_list = generate_dcip_sources_line(
        survey_type,
        data_type,
        dimension_type,
        end_locations,
        topo_2d,
        num_rx_per_src,
        station_separation,
    )

    # Define survey
    survey = dc.survey.Survey(source_list, survey_type=survey_type) # Tipo de arreglo; # electrodos; Total de datos

    if surveyinfo["data_type"] == "apparent_resistivity":
        survey.set_geometric_factor()

    l = np.diff(end_locations)[0] # Longitud del arreglo

    dh = l/ncol # Tamaño de celda x 
    dz = surveyinfo["depth"]/nrow # Tamaño de celda y
    hx = [(dh, ncol)] # [(Tamaño de celda, número de celdas)]
    hz = [(dz, nrow)]
    mesh = TensorMesh([hx, hz], "0N") # 0: centrado en el origen; N: Inicio de eje negativo: C: 0 como centro 

    model = np.log(model.flatten())

    mapping = maps.ExpMap(mesh)

    simulation_dc = dc.Simulation2DNodal(
        mesh, rhoMap=mapping, solver=Solver, survey=survey, storeJ=True, nky=12
    )

    dc_data = simulation_dc.make_synthetic_data(model, relative_error=0, add_noise=False)

    return mapping, survey, mesh, simulation_dc, dc_data


In [ ]:
model = np.load(r"C:\Users\eaqm1\Downloads\model.npy")

surveyinfo = dict(nElec = 26, sep = 8.0, pi=0, depth= 22.,  
                  typ_survey="dipole-dipole", dim="2D", 
                  data_type="volt", nlines=10)

mapping, survey, mesh, simulation_dc, dc_data = Forward(model, surveyinfo)

In [ ]:
def Invert(floor = 1e-3, percent_std = 0.05, alpha_s = 1e-3, 
           alpha_x=1, alpha_y=1, iter=20, b0_ratio = 1e2, use_target = False,
           CF=2., CR=1):

    floor = floor
    percent_std = percent_std

    dc_data.standard_deviation = floor + percent_std * np.abs(dc_data.dobs)

    mref = np.log(np.median(apparent_resistivity_from_voltage(survey,dc_data.dobs)))*np.ones(mesh.nC)
    dmisfit = data_misfit.L2DataMisfit(data=dc_data, simulation=simulation_dc)
    reg = regularization.WeightedLeastSquares(
        mesh, alpha_s=alpha_s, alpha_x=alpha_x, alpha_y=alpha_y, reference_model=mref*np.ones(mesh.nC)
    )

    opt = optimization.InexactGaussNewton(maxIter=iter, maxIterCG=iter)
    opt.remember("xc")
    base_inv = inverse_problem.BaseInvProblem(dmisfit, reg, opt)

    beta_est = directives.BetaEstimate_ByEig(beta0_ratio=b0_ratio)
    target = directives.TargetMisfit(chifact=1)
    save = SaveInversionProgress()
    directives_list = [beta_est, save]

    if use_target is True:
        directives_list.append(target)

    beta_schedule = directives.BetaSchedule(coolingFactor=CF, coolingRate=CR)
    directives_list.append(beta_schedule)

    inv = inversion.BaseInversion(base_inv, directiveList=directives_list)
    model_recovered = inv.run(mref)
    inv_result = save.inversion_results

    return inv_result 

inv_result = Invert()

In [ ]:
def plot_model(
    model=None, ax=None, 
    colorbar=True
):
    """
    A plotting function for our recovered model 
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    
    out = mesh.plot_image(
        mapping * model, pcolorOpts={'norm':LogNorm(), 'cmap':'Spectral'}, ax=ax,
        # clim=clim
    )
    ax.set_ylabel('Elevation (m)')
    ax.set_xlabel('Easting (m)')
    ax.set_aspect(1.5)  # some vertical exxageration
    
    if colorbar is True:
        cb = plt.colorbar(out[0], fraction=0.05, orientation='horizontal', ax=ax, pad=0.25)
        cb.set_label("Resistivity ($\Omega$m)")
    
    return ax

def plot_tikhonov_curve(inversion_results, ax=None):
    """
    Plot the Tikhonov curve: Phi_d as a function of Phi_m
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    
    ax.plot(inv_result["phi_m"], inv_result["phi_d"], "-oC6", label="$\Phi_d$")
    ax.axhline(survey.nD / 2, linestyle="--", color="k", label="$\Phi_d^*$")
    ax.axhline(survey.nD / 2, linestyle="-.", color="k", label="$\chi\Phi_d^*$")
    ax.set_ylabel("$\Phi_d$")
    ax.set_xlabel(x)
    
    ax.set_xlabel("$\Phi_m$")
    ax.set_ylabel("$\Phi_d$")
    
    ax.legend(loc=1)
    return ax

def plot_normalized_misfit(iteration=None, observed_data=dc_data, ax=None):
    """
    Plot the normalized data misfit
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1)
        
    if iteration is None:
        dpred = inv_result["dpred"][-1]
    else:
        dpred = inv_result["dpred"][iteration]
        
    normalized_misfit = (dpred - observed_data.dobs)/observed_data.standard_deviation
    out = dc.utils.plot_pseudosection(
        observed_data, dobs=normalized_misfit, data_type="misfit",
        plot_type="contourf", data_location=True, ax=ax, 
        cbar_opts={"pad":0.25}
    )

    ax.set_title("normalized misfit")
    ax.set_yticklabels([])
    ax.set_aspect(1.5)  # some vertical exxageration
    ax.set_xlabel("Northing (m)")
    ax.set_ylabel("n-spacing")

    cb_axes = plt.gcf().get_axes()[-1]
    cb_axes.set_xlabel('normalized misfit')
    return ax

def plot_results_at_iteration(iteration=0):
    """
    A function to plot inversion results as a function of iteration 
    """
    fig = plt.figure(figsize=(12, 8))
    spec = gridspec.GridSpec(ncols=6, nrows=2, figure=fig)

    ax_tikhonov = fig.add_subplot(spec[:, :2])
    ax_misfit = fig.add_subplot(spec[0, 2:])
    ax_model = fig.add_subplot(spec[1, 2:])

    plot_tikhonov_curve(inv_result, ax=ax_tikhonov)
    ax_tikhonov.plot(
        inv_result["phi_m"][iteration], inv_result["phi_d"][iteration], 
        'ks', ms=10
    )
    ax_tikhonov.set_title(f"iteration {iteration}")
    
    plot_normalized_misfit(iteration=iteration, ax=ax_misfit)

    plot_model(inv_result["model"][iteration], ax=ax_model, )
    # make the tikhonov plot square
    ax_tikhonov.set_aspect(1./ax_tikhonov.get_data_ratio())

    plt.tight_layout()

In [ ]:
inversion_results_app = ipywidgets.interact(
    plot_results_at_iteration,
    iteration = ipywidgets.IntSlider(min=0, max=inv_result["iteration"][-1]-1, value=0)
)
inversion_results_app